In [1]:
import abc
import pandas as pd
from sqlalchemy.engine import Engine
from utils.utils import easyConnect, upsert_to_mysql
import numpy as np

08-21 08:50:09 | INFO    | utils.logger_config:_setup_logger | 📝 日志系统启动成功
08-21 08:50:09 | INFO    | utils.logger_config:_setup_logger | 📁 主日志文件: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos\logs\20250821-1.log
08-21 08:50:09 | INFO    | utils.logger_config:_setup_logger | 🔴 错误日志文件: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos\logs\20250821-1_error.log
08-21 08:50:09 | INFO    | utils.logger_config:_setup_logger | 📂 项目根目录: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos


In [15]:
df = pd.read_excel('../assets/test.xlsx')

In [16]:
df

,ts_code,trade_date,data1,data2
0,000001.SZ,2024-08-18,2,6
1,000001.SZ,2024-08-19,4,12
2,000001.SZ,2024-08-20,5,15
3,000001.SZ,2024-08-21,6,12
4,000002.SZ,2024-08-18,3,6
5,000002.SZ,2024-08-19,12,12
6,000002.SZ,2024-08-20,6,12
7,000002.SZ,2024-08-21,8,4
8,000002.SZ,2024-08-22,9,3
9,000002.SZ,2024-08-23,10,15


In [4]:
def avg(df, window: int, column: str, how: str):
    """
    在 DataFrame 中添加一列移动平均值
    :param df: DataFrame
    :param window: 移动窗口大小
    :param column: 要计算的列名
    :param how: 平均方式, 'simple' | 'weighted' | 'exponential'
    :return: None (直接修改 df)
    """
    how = how.lower()
    new_col = f"{column}_{how}_ma{window}"

    if how == 'simple':
        df[new_col] = df[column].rolling(window=window, min_periods=1).mean()

    elif how == 'weighted':
        def weighted_moving_average(x):
            weights = np.arange(1, len(x) + 1)  # 权重 1,2,3,...,window
            return np.dot(x, weights) / weights.sum()
        df[new_col] = df[column].rolling(window=window, min_periods=1).apply(weighted_moving_average, raw=True)

    elif how == 'exponential':
        df[new_col] = df[column].ewm(span=window, adjust=False).mean()

    else:
        raise ValueError("how 必须是 'simple'、'weighted' 或 'exponential'")

In [5]:
avg(df, window=2, column='data1', how='simple')
df

,ts_code,trade_date,data1,data2,data1_simple_ma2
0,000001.SZ,2024-08-18,2,6,2.0
1,000001.SZ,2024-08-19,4,12,3.0
2,000001.SZ,2024-08-20,5,15,4.5
3,000001.SZ,2024-08-21,6,12,5.5
4,000002.SZ,2024-08-18,3,6,4.5
5,000002.SZ,2024-08-19,12,12,7.5
6,000002.SZ,2024-08-20,6,12,9.0
7,000002.SZ,2024-08-21,8,4,7.0
8,000002.SZ,2024-08-22,9,3,8.5
9,000002.SZ,2024-08-23,10,15,9.5


In [17]:
def add(df, add1:str, add2:str, result:str=""):
    if result:
        df[result] = df[add1]+df[add2]
    else:
        print(f'{add1}_add_{add2}')
        df[f'{add1}_add_{add2}'] = df[add1]+df[add2]


In [18]:
add(df, "data1","data2")

data1_add_data2


In [19]:
df

,ts_code,trade_date,data1,data2,data1_add_data2
0,000001.SZ,2024-08-18,2,6,8
1,000001.SZ,2024-08-19,4,12,16
2,000001.SZ,2024-08-20,5,15,20
3,000001.SZ,2024-08-21,6,12,18
4,000002.SZ,2024-08-18,3,6,9
5,000002.SZ,2024-08-19,12,12,24
6,000002.SZ,2024-08-20,6,12,18
7,000002.SZ,2024-08-21,8,4,12
8,000002.SZ,2024-08-22,9,3,12
9,000002.SZ,2024-08-23,10,15,25


In [26]:
def sub(df, sub1:str, sub2:str, result:str=""):
    if result:
        df[result] = df[sub1]-df[sub2]
    else:
        print(f'{sub1}_sub_{sub2}')
        df[f'{sub1}_sub_{sub2}'] = df[sub1]-df[sub2]

In [29]:
sub(df, "data1","data2")

In [30]:
df

,ts_code,trade_date,data1,data2,data1_add_data2,data1_sub_data2,ttt
0,000001.SZ,2024-08-18,2,6,8,-4,-4
1,000001.SZ,2024-08-19,4,12,16,-8,-8
2,000001.SZ,2024-08-20,5,15,20,-10,-10
3,000001.SZ,2024-08-21,6,12,18,-6,-6
4,000002.SZ,2024-08-18,3,6,9,-3,-3
5,000002.SZ,2024-08-19,12,12,24,0,0
6,000002.SZ,2024-08-20,6,12,18,-6,-6
7,000002.SZ,2024-08-21,8,4,12,4,4
8,000002.SZ,2024-08-22,9,3,12,6,6
9,000002.SZ,2024-08-23,10,15,25,-5,-5


In [31]:
def mul(df, mul1:str, mul2:str, result:str=""):
    if result:
        df[result] = df[mul1]*df[mul2]
    else:
        print(f'{mul1}_mul_{mul2}')
        df[f'{mul1}_mul_{mul2}'] = df[mul1]*df[mul2]

def div(df, div1:str, div2:str, result:str=""):
    if result:
        df[result] = df[div1]/df[div2]
    else:
        print(f'{div1}_div_{div2}')
        df[f'{div1}_div_{div2}'] = df[div1]/df[div2]


In [32]:
mul(df, "data1","data2")

data1_mul_data2


In [33]:
div(df, "data1","data2")

data1_div_data2


In [35]:
df

,ts_code,trade_date,data1,data2,data1_add_data2,data1_sub_data2,ttt,data1_mul_data2,data1_div_data2
0,000001.SZ,2024-08-18,2,6,8,-4,-4,12,0.333333
1,000001.SZ,2024-08-19,4,12,16,-8,-8,48,0.333333
2,000001.SZ,2024-08-20,5,15,20,-10,-10,75,0.333333
3,000001.SZ,2024-08-21,6,12,18,-6,-6,72,0.500000
4,000002.SZ,2024-08-18,3,6,9,-3,-3,18,0.500000
5,000002.SZ,2024-08-19,12,12,24,0,0,144,1.000000
6,000002.SZ,2024-08-20,6,12,18,-6,-6,72,0.500000
7,000002.SZ,2024-08-21,8,4,12,4,4,32,2.000000
8,000002.SZ,2024-08-22,9,3,12,6,6,27,3.000000
9,000002.SZ,2024-08-23,10,15,25,-5,-5,150,0.666667


In [ ]:
def changedate(origin_date, days:int):
    return (pd.to_datetime(origin_date)+pd.DateOffset(days=days)).strftime('%Y-%m-%d')

Timestamp('2025-08-31 00:00:00')

In [3]:
import pandas as pd
a = {'a':1,'b':2,'c':3}


KeyError: 'd'

In [7]:
import tushare as ts
from utils.utils import easyPro
easyPro()
ts.pro_bar(ts_code='000300.SH',adj='qfq', start_date='20250501',end_date='20250801')

In [2]:
a = "1,2,3"
l = a.split(",")

In [ ]:
b = [1]
b.